In [4]:
%pylab
%matplotlib inline
import SimMLA.fftpack as simfft
import SimMLA.grids   as grids

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


# Simulation of partially coherent beams
The rotating diffuser in the illuminator setup has the effect of randomizing the phase of the incident laser beam. This randomization results in a decrease in the spatial coherence of the beam, which smears out the array resulting from the coherent beam illumination.

To simulate the effect of partial spatial coherence, I am employing a simulation strategy detailed in [Xiao and Voelz, Optics Express 14, 6986-6992 (2006)](https://www.osapublishing.org/oe/fulltext.cfm?uri=oe-14-16-6986&id=96179). The strategy employed in this paper is to firstgenerate a random phase screen that satisfies the Gaussian-Schell model. The input field is multipled by this phase screen and then propagated through the illuminator to the sample. Another phase screen is generated and the process is repeated a large number of times. The average irradiance on the sample becomes the average over these realizations.

In [5]:
# Create a grid to sample the input field and build the coherence function on
numLenslets  = 21    # Must be odd; corresponds to the number of lenslets in one dimension
lensletSize  = 500   # microns
focalLength  = 13700 # microns

wavelength   = 0.642 # microns

subgridSize  = 10001                     # Number of grid (or lattice) sites for a single lenslet
physicalSize = numLenslets * lensletSize # The full extent of the MLA

# dim = 1 makes the grid 1D
grid = grids.GridArray(numLenslets, subgridSize, physicalSize, wavelength, focalLength, dim = 1)

In [18]:
# Define the amplitude variation and correlation length of the phase screen
sigma_r = 100 * np.sqrt(4 * np.pi) # microns
sigma_f = 10 # microns

GS_Criterion = sigma_r**2 / 4 / np.pi / sigma_f**2

In [17]:
GS_Criterion

99.999999999999986

16.0